## 尝试整数线性规划

In [34]:
import numpy as np
import pulp
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load the data from the CSV file
file_path = './data/food_nutrition.csv'
nutrition_data = pd.read_csv(file_path)

# Display the data to understand its structure and contents
food_names = nutrition_data['name'].tolist()
nutrition_data.drop(['unit'], axis=1, inplace=True)       # drop the `unit` column
nutrition_data = nutrition_data.reset_index(drop=True)

# Separate the nutrition data for the foods and the lower/upper bounds
food_nutrition = nutrition_data.iloc[2:]
lower_bounds = nutrition_data.iloc[0, 1:]
upper_bounds = nutrition_data.iloc[1, 1:]

# Protein的上下限与WEIGHT有关 => WEIGHT * [.8, 2]
# Fat的上下限与energy有关 => energy * [0.2, 0.35]
WEIGHT = 60             # 60kg body weight
lower_bounds[2] *= WEIGHT
upper_bounds[2] *= WEIGHT
lower_bounds[3] = lower_bounds[1] * 0.2
upper_bounds[3] = upper_bounds[1] * 0.35


YEAR = 20

# 示例数据
# F: 食物中的营养成分矩阵
# L: 营养成分的最小需求
# U: 营养成分的最大限制
# 假设 F 是食物和营养成分的矩阵，L 和 U 分别是营养成分的下界和上界
F = food_nutrition.drop(['name'], axis=1).to_numpy() * 365 
print('F: ', F.shape)

SCALAR1, SCALAR2 = 0.75, 1.1    # 用来调整上下界
L = lower_bounds.to_numpy() * SCALAR1 * YEAR * 365  
U = upper_bounds.to_numpy() * SCALAR2 * YEAR * 365
print('L and U: ', L.shape, U.shape)


# 创建线性规划问题
lp_prob = pulp.LpProblem("Nutrition_Optimization", pulp.LpMinimize)

# 创建变量
n_foods = F.shape[0]
food_vars = [pulp.LpVariable(f'food_{food_names[i+2]}', lowBound=0, cat='Integer') for i in range(n_foods)]

# 目标函数：最小化食物数量
lp_prob += pulp.lpSum(food_vars)

# 添加营养成分的约束
for j in range(F.shape[1]):
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) >= L[j]
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) <= U[j]

# 求解问题
lp_prob.solve(pulp.PULP_CBC_CMD(msg=0))


# 打印结果
if pulp.LpStatus[lp_prob.status] == 'Optimal':
    print("找到最优解")
    for var in food_vars:
        print(int(var.varValue), "个", var.name)
else:
    print("无法找到最优解")

F:  (25, 24)
L and U:  (24,) (24,)
找到最优解
0 个 food_watermelon
0 个 food_apple
0 个 food_banana
0 个 food_avocado
0 个 food_broccoli
71 个 food_pink_salmon
0 个 food_Chicken;_broilers_or_fryers;_leg;_meat_and_skin;_raw
0 个 food_Beef;_ribeye_cap_steak;_boneless;_separable_lean_only;_trimmed_to_0"_fat;_choice;_raw
0 个 food_arugula
2 个 food_Bread;_whole_wheat;_commercially_prepared;_toasted
0 个 food_egg
63 个 food_Corn;_sweet;_white;_raw
0 个 food_Alcoholic_beverage;_beer;_light
1 个 food_Alcoholic_beverage;_distilled;_vodka;_80_proof	
1 个 food_Beverages;_coffee;_brewed;_espresso;_restaurant_prepared
0 个 food_Rice_noodles;_cooked
0 个 food_Rice_and_vermicelli_mix;_rice_pilaf_flavor;_unprepared
63 个 food_Beverages;_rice_milk;_unsweetened
0 个 food_Yogurt;_plain;_whole_milk
17 个 food_Tofu;_dried_frozen_(koyadofu)
0 个 food_Bread;_cornbread;_dry_mix;_enriched_(includes_corn_muffin_mix)
27 个 food_Oil;_corn;_peanut;_and_olive
17 个 food_mango
8 个 food_cilantro_leaves;_raw
0 个 food_sugar
